In [1]:
import pandas as pd

In [3]:
em=pd.read_csv(r'C:\Users\HP\Downloads\emotion_nlp.csv')

In [5]:
em.head()

,Text,Emotions
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [39]:
em.rename(columns={'Emotions':'Y','Text':'X'},inplace=True)
em.head()

,X,Y
0,i didnt feel humiliated,1
1,i can go from feeling so hopeless to so damned...,1
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1


In [9]:
em.Y.unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [11]:
em.Y.replace({'sadness':1, 'anger':1, 'love':2, 'surprise':2, 'fear':1, 'joy':2},inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_14060\2141937404.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  em.Y.replace({'sadness':1, 'anger':1, 'love':2, 'surprise':2, 'fear':1, 'joy':2},inplace=True)


In [15]:
import nltk
from nltk.corpus import stopwords

In [19]:
em['X']=em['X'].str.lower()

In [17]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
l1=string.punctuation
l2=stopwords.words("english")

In [25]:
def text_process(mess):            ### creating a function

    """                                                        ## a docstring

    1. remove the punctuation

    2. remove the stopwords

    3. return the list of clean textwords

    """

    nopunc = [char for char in mess if char not in l1]

    nopunc = "".join(nopunc)

    return [ word for word in nopunc.split() if word not in l2]

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer= CountVectorizer(analyzer=text_process).fit(em['X']) 

In [33]:
em_bow= bow_transformer.transform(em.X)

In [35]:
em_bow.shape

(17999, 16035)

In [41]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(em_bow, em.Y,random_state=123,test_size = .2)

In [43]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,recall_score,f1_score

In [47]:
from sklearn.linear_model import LogisticRegression        
lr=LogisticRegression()
lr.fit(train_x,train_y) 

LogisticRegression()

In [49]:
pred_log = lr.predict(test_x)
tab1 = confusion_matrix(test_y,pred_log)
tab1

array([[1885,   82],
       [  59, 1574]], dtype=int64)

In [51]:
print(classification_report(test_y,pred_log))

              precision    recall  f1-score   support

           1       0.97      0.96      0.96      1967
           2       0.95      0.96      0.96      1633

    accuracy                           0.96      3600
   macro avg       0.96      0.96      0.96      3600
weighted avg       0.96      0.96      0.96      3600



In [53]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(train_x,train_y)

MultinomialNB()

In [55]:
pred_nb= nb.predict(test_x)
tab1 = confusion_matrix(test_y, pred_nb)
tab1

array([[1872,   95],
       [ 149, 1484]], dtype=int64)

In [57]:
print(classification_report(test_y, pred_nb))

              precision    recall  f1-score   support

           1       0.93      0.95      0.94      1967
           2       0.94      0.91      0.92      1633

    accuracy                           0.93      3600
   macro avg       0.93      0.93      0.93      3600
weighted avg       0.93      0.93      0.93      3600



In [59]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(train_x , train_y)

DecisionTreeClassifier()

In [61]:
pred_dt = DT.predict(test_x)
tab1 = confusion_matrix(test_y, pred_dt)
tab1

array([[1768,  199],
       [ 100, 1533]], dtype=int64)

In [63]:
print(classification_report(test_y, pred_dt))

              precision    recall  f1-score   support

           1       0.95      0.90      0.92      1967
           2       0.89      0.94      0.91      1633

    accuracy                           0.92      3600
   macro avg       0.92      0.92      0.92      3600
weighted avg       0.92      0.92      0.92      3600



In [65]:
from sklearn.ensemble import RandomForestClassifier
rfc= RandomForestClassifier()
rfc.fit(train_x,train_y)

RandomForestClassifier()

In [66]:
pred_rfc = rfc.predict(test_x)
tab1 = confusion_matrix(test_y, pred_rfc)
tab1

array([[1817,  150],
       [  55, 1578]], dtype=int64)

In [67]:
print(classification_report(test_y, pred_rfc))

              precision    recall  f1-score   support

           1       0.97      0.92      0.95      1967
           2       0.91      0.97      0.94      1633

    accuracy                           0.94      3600
   macro avg       0.94      0.95      0.94      3600
weighted avg       0.94      0.94      0.94      3600

